# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 19 2022 9:31AM,246692,10,SONSB0001942,"Nextsource Biotechnology, LLC",Released
1,Sep 19 2022 9:28AM,246691,12,HH-36963,Hush Hush,Released
2,Sep 19 2022 9:28AM,246691,12,HH-36964,Hush Hush,Released
3,Sep 19 2022 9:28AM,246691,12,HH-36965,Hush Hush,Released
4,Sep 19 2022 9:28AM,246691,12,HH-36966,Hush Hush,Released
5,Sep 19 2022 9:28AM,246691,12,HH-36967,Hush Hush,Released
6,Sep 19 2022 9:28AM,246691,12,HH-36968,Hush Hush,Released
7,Sep 19 2022 9:28AM,246691,12,HH-36969,Hush Hush,Released
8,Sep 19 2022 9:28AM,246691,12,HH-36970,Hush Hush,Released
9,Sep 19 2022 9:28AM,246691,12,HH-36971,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,246684,Released,48
12,246688,Released,20
13,246689,Released,1
14,246691,Released,41
15,246692,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246684,NaN,48.0
246688,NaN,20.0
246689,NaN,1.0
246691,NaN,41.0
246692,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246672,6.0,1.0
246678,1.0,9.0
246679,42.0,18.0
246680,0.0,1.0
246681,0.0,56.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246672,6,1
246678,1,9
246679,42,18
246680,0,1
246681,0,56


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246672,6,1
1,246678,1,9
2,246679,42,18
3,246680,0,1
4,246681,0,56


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246672,6,1
1,246678,1,9
2,246679,42,18
3,246680,,1
4,246681,,56


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 19 2022 9:31AM,246692,10,"Nextsource Biotechnology, LLC"
1,Sep 19 2022 9:28AM,246691,12,Hush Hush
42,Sep 19 2022 9:17AM,246689,10,"Citieffe, Inc."
43,Sep 19 2022 9:17AM,246688,10,ISDIN Corporation
63,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation
111,Sep 19 2022 8:44AM,246683,10,ISDIN Corporation
170,Sep 19 2022 8:35AM,246682,10,Emerginnova
176,Sep 19 2022 8:33AM,246681,10,ISDIN Corporation
232,Sep 19 2022 8:29AM,246679,10,ISDIN Corporation
292,Sep 19 2022 8:25AM,246680,16,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 19 2022 9:31AM,246692,10,"Nextsource Biotechnology, LLC",,1
1,Sep 19 2022 9:28AM,246691,12,Hush Hush,,41
2,Sep 19 2022 9:17AM,246689,10,"Citieffe, Inc.",,1
3,Sep 19 2022 9:17AM,246688,10,ISDIN Corporation,,20
4,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,,48
5,Sep 19 2022 8:44AM,246683,10,ISDIN Corporation,,59
6,Sep 19 2022 8:35AM,246682,10,Emerginnova,2,4
7,Sep 19 2022 8:33AM,246681,10,ISDIN Corporation,,56
8,Sep 19 2022 8:29AM,246679,10,ISDIN Corporation,42,18
9,Sep 19 2022 8:25AM,246680,16,Sartorius Stedim North America,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 9:31AM,246692,10,"Nextsource Biotechnology, LLC",1,
1,Sep 19 2022 9:28AM,246691,12,Hush Hush,41,
2,Sep 19 2022 9:17AM,246689,10,"Citieffe, Inc.",1,
3,Sep 19 2022 9:17AM,246688,10,ISDIN Corporation,20,
4,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,48,
5,Sep 19 2022 8:44AM,246683,10,ISDIN Corporation,59,
6,Sep 19 2022 8:35AM,246682,10,Emerginnova,4,2
7,Sep 19 2022 8:33AM,246681,10,ISDIN Corporation,56,
8,Sep 19 2022 8:29AM,246679,10,ISDIN Corporation,18,42
9,Sep 19 2022 8:25AM,246680,16,Sartorius Stedim North America,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 9:31AM,246692,10,"Nextsource Biotechnology, LLC",1,
1,Sep 19 2022 9:28AM,246691,12,Hush Hush,41,
2,Sep 19 2022 9:17AM,246689,10,"Citieffe, Inc.",1,
3,Sep 19 2022 9:17AM,246688,10,ISDIN Corporation,20,
4,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,48,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 9:31AM,246692,10,"Nextsource Biotechnology, LLC",1,NaN
1,Sep 19 2022 9:28AM,246691,12,Hush Hush,41,NaN
2,Sep 19 2022 9:17AM,246689,10,"Citieffe, Inc.",1,NaN
3,Sep 19 2022 9:17AM,246688,10,ISDIN Corporation,20,NaN
4,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,48,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 9:31AM,246692,10,"Nextsource Biotechnology, LLC",1,0.0
1,Sep 19 2022 9:28AM,246691,12,Hush Hush,41,0.0
2,Sep 19 2022 9:17AM,246689,10,"Citieffe, Inc.",1,0.0
3,Sep 19 2022 9:17AM,246688,10,ISDIN Corporation,20,0.0
4,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,48,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2220156,216,45.0
12,246691,41,0.0
16,246680,1,0.0
19,246672,1,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2220156,216,45.0
1,12,246691,41,0.0
2,16,246680,1,0.0
3,19,246672,1,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,216,45.0
1,12,41,0.0
2,16,1,0.0
3,19,1,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,216.0
1,12,Released,41.0
2,16,Released,1.0
3,19,Released,1.0
4,10,Executing,45.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19
Status,,,,
Executing,45.0,0.0,0.0,6.0
Released,216.0,41.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19
0,Executing,45.0,0.0,0.0,6.0
1,Released,216.0,41.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19
0,Executing,45.0,0.0,0.0,6.0
1,Released,216.0,41.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()